In [1]:
import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, cross_val_score, GridSearchCV, KFold
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from imblearn.over_sampling import SMOTE, SVMSMOTE, RandomOverSampler
from imblearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2, SelectFpr, f_regression, f_classif
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, KBinsDiscretizer
from collections import Counter
import seaborn as sns

SEED = 5678
np.random.seed(SEED)

In [3]:
train = pd.read_csv(r'C:\Users\Me\Kaggle\Titanic_revisited\data\train.csv')
test = pd.read_csv(r'C:\Users\Me\Kaggle\Titanic_revisited\data\test.csv')
print(train.shape, test.shape)

(891, 12) (418, 11)


# SOLS Classifier
> This is a custom classifier created using Scikit Learn's project template for custom estimators.
> It uses statsmodels OLS for the initial fit. Predict then uses the same OLS model to make a 
> prediction. The median of the predictions is subtracted from the prediction to create an estimated 
> residual. The estimated residual is then divided by the standard deviation of the residuals from the fit
> to create an estimated studentized residual. These studentized residuals are then tested against the
> hyperparameter "threshold" to determine the label. All  values greater than or equal to the threshold are 
> labeled True, and values less than the threshold are labeled False.

In [4]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import euclidean_distances
import pandas as pd
from statsmodels.stats.outliers_influence import OLSInfluence
from statsmodels.regression.linear_model import OLS

class SOLSClassifier(BaseEstimator, ClassifierMixin ):
    
    def __init__(self, threshold = 'threshold', est_method = 'est_method'):
        
        #decision threshold of studentized residuals
        self.threshold = threshold 
        self.est_method = est_method
        np.random.seed(SEED)
        
    def fit(self, X, y):
        
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        
        # Store the classes seen during fit
        self.classes_ = unique_labels(y) 
        
        #convert to df
        self.X_ = pd.DataFrame(X)
        self.y_ = pd.DataFrame(y)         
        
        #Fit OLS model
        self.ols_mod = OLS(endog = self.y_, exog = self.X_)
        self.ols_result = self.ols_mod.fit()
        
        # Return the classifier
        return self

    def predict(self, X):
        
        # Check if fit had been called
        check_is_fitted(self, ['X_', 'y_'])
        
        # Input validation
        X = check_array(X)
        X_n = pd.DataFrame(X)
        
        #OLS prediction       
        prediction = self.ols_result.predict(X_n)        
        
        #calculate outlier and influence measures for OLS result
        inf = OLSInfluence(self.ols_result)
        
        #Staandard Deviation of studentized residuals
        std = inf.resid_std
        
        """
        Subtract the median of the predictions from the predictions to create an estimated residual.
        Then divide the estiamted residual by the by the estimated standard deviation, the
        standard deviation of the residuals from training, to create an estimated studentized residual.
           
        """ 
        # estimated residual
        estimated_residual = prediction - np.nanmedian(prediction)
        
        #estiamted studentized residual
        if self.est_method == 'mean':
            stud_res = estimated_residual/np.nanmean(std)    #estimate using mean
        if self.est_method == 'median':
            stud_res = prediction/np.nanmedian(std)         #estimate using median
        
        #create predictions based on the threshold
        self.preds = []        
        for res in stud_res:
            if res >= self.threshold:    
                #self.preds.append(True)
                self.preds.append(1)
            else:
                #self.preds.append(False)
                self.preds.append(0)
                
        return self.preds

# Functions

In [5]:
PERF_FORMAT_STRING = "\
\tAccuracy: {:>0.{display_precision}f}\tPrecision: {:>0.{display_precision}f}\t\
Recall: {:>0.{display_precision}f}\tF1: {:>0.{display_precision}f}\tF2: {:>0.{display_precision}f}"
RESULTS_FORMAT_STRING = "\tTotal predictions: {:4d}\tTrue positives: {:4d}\tFalse positives: {:4d}\
\tFalse negatives: {:4d}\tTrue negatives: {:4d}"

#Get classifier metrics, based on DAND
def test_classifier(clf, dataset, feature_list, folds = 20):
    data = featureFormat(dataset, feature_list, sort_keys = True)
    #data = np.array(dataset)
    labels, features = targetFeatureSplit(data)
    sss = StratifiedShuffleSplit(n_splits = folds, random_state = SEED)
    cv = sss.split(features, labels)
    
    true_negatives = 0
    false_negatives = 0
    true_positives = 0
    false_positives = 0
    for train_idx, test_idx in cv: 
        features_train = []
        features_test  = []
        labels_train   = []
        labels_test    = []
        for ii in train_idx:
            features_train.append( features[ii] )
            labels_train.append( labels[ii] )
        for jj in test_idx:
            features_test.append( features[jj] )
            labels_test.append( labels[jj] )
        
        ### fit the classifier using training set, and test on test set
        clf.fit(features_train, labels_train)
        predictions = clf.predict(features_test)
        for prediction, truth in zip(predictions, labels_test):
            if prediction == 0 and truth == 0:
                true_negatives += 1
            elif prediction == 0 and truth == 1:
                false_negatives += 1
            elif prediction == 1 and truth == 0:
                false_positives += 1
            elif prediction == 1 and truth == 1:
                true_positives += 1
            else:
                print ("Warning: Found a predicted label not == 0 or 1.")
                print ("All predictions should take value 0 or 1.")
                print ("Evaluating performance for processed predictions:")
                break
    try:
        total_predictions = true_negatives + false_negatives + false_positives + true_positives
        accuracy = 1.0*(true_positives + true_negatives)/total_predictions
        precision = 1.0*true_positives/(true_positives+false_positives)
        recall = 1.0*true_positives/(true_positives+false_negatives)
        f1 = 2.0 * true_positives/(2*true_positives + false_positives+false_negatives)
        f2 = (1+2.0*2.0) * precision*recall/(4*precision + recall)
        print (clf)
        print (PERF_FORMAT_STRING.format(accuracy, precision, recall, f1, f2, display_precision = 5))
        print (RESULTS_FORMAT_STRING.format(total_predictions, true_positives, false_positives, false_negatives, true_negatives))
        print ("")
    except:
        print ("Got a divide by zero when trying out:", clf)
        print ("Precision or recall may be undefined due to a lack of true positive predicitons.")


#convert dictionary to numpy array of features, from DAND       
def featureFormat( dictionary, features, remove_NaN=True, remove_all_zeroes=True, remove_any_zeroes=False, sort_keys = False):
    
    return_list = []

    # Key order - first branch is for Python 3 compatibility on mini-projects,
    # second branch is for compatibility on final project.
    if isinstance(sort_keys, str):
        import pickle
        keys = pickle.load(open(sort_keys, "rb"))
    elif sort_keys:
        keys = sorted(dictionary.keys())
    else:
        keys = dictionary.keys()

    for key in keys:
        tmp_list = []
        for feature in features:
            try:
                dictionary[key][feature]
            except KeyError:
                print ("error: key ", feature, " not present")
                return
            value = dictionary[key][feature]
            if value=="NaN" and remove_NaN:
                value = 0
            tmp_list.append( float(value) )

        # Logic for deciding whether or not to add the data point.
        append = True
        # exclude 'poi' class as criteria.
        if features[0] == 'poi':
            test_list = tmp_list[1:]
        else:
            test_list = tmp_list
        ### if all features are zero and you want to remove
        ### data points that are all zero, do that here
        if remove_all_zeroes:
            append = False
            for item in test_list:
                if item != 0 and item != "NaN":
                    append = True
                    break
        ### if any features for a given data point are zero
        ### and you want to remove data points with any zeroes,
        ### handle that here
        if remove_any_zeroes:
            if 0 in test_list or "NaN" in test_list:
                append = False
        ### Append the data point if flagged for addition.
        if append:
            return_list.append( np.array(tmp_list) )

    return np.array(return_list)

# prepares data for tester returns list and dict 
def tester_prep(dfn):
    features_list = dfn.columns.values
    data_dict = dfn.to_dict('index')
    return features_list, data_dict

#split FIRST row as labels, from DAND
def targetFeatureSplit( data ):
    target = []
    features = []
    for item in data:
        target.append( item[0] )
        features.append( item[1:] )
    return target, features

#Custom Round
def custom_round(x, base=5):
    return base * round(float(x)/base)

#fits model to kfolds and returns column with predictions
def fit_folds(model, X_train, y_train,  n_folds):
    
    new_col = np.zeros((X_train.shape[0], 1))
    kfold = KFold( n_splits = n_folds, shuffle=True, random_state=SEED) 
        
    for train_index, test_index in kfold.split(X_train):  
        Xt, Xv = X_train.iloc[train_index], X_train.iloc[test_index]
        yt, yv = y_train.iloc[train_index], y_train.iloc[test_index]
        model.fit(Xt, yt)
        pred = np.column_stack([model.predict(Xv)])        
        new_col[test_index] = pred
                        
    return(new_col)

# New Data

In [6]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
train.isna().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool

In [8]:
test.isna().any()

PassengerId    False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare            True
Cabin           True
Embarked       False
dtype: bool

In [9]:
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [10]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [11]:
#Split to X,y
X = train.copy()
y = train['Survived'].copy()
X.drop(['Survived'], axis = 1, inplace = True)
print(y.shape, X.shape)

(891,) (891, 11)


In [12]:
X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
#Deck
X['Deck'] = X['Cabin'].apply(lambda s: s[0] if pd.notnull(s) else 'M')
test['Deck'] = test['Cabin'].apply(lambda s: s[0] if pd.notnull(s) else 'M')
X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,M
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,M
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,C
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,M


In [14]:
# Drop Columns
PassengerId =np.array(test["PassengerId"]).astype(int)
X.drop(labels = [ "PassengerId",  "Cabin"], axis = 1, inplace = True)
test.drop(labels = [ "PassengerId",  "Cabin"], axis = 1, inplace = True)

In [15]:
#ticket frequency
X['Ticket_Frequency'] = X.groupby('Ticket')['Ticket'].transform('count')
test['Ticket_Frequency'] = test.groupby('Ticket')['Ticket'].transform('count')

X.drop(labels = [ "Ticket"], axis = 1, inplace = True)
test.drop(labels = [ "Ticket"], axis = 1, inplace = True)

In [16]:
X.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Deck,Ticket_Frequency
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,M,1
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,C,1
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,M,1
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,C,2
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,M,1


In [17]:
#Extract title before dropping name
dataset_title = [i.split(",")[1].split(".")[0].strip() for i in X["Name"]]
X["Title"] = pd.Series(dataset_title)
X["Title"].head()

dataset_title = [i.split(",")[1].split(".")[0].strip() for i in test["Name"]]
test["Title"] = pd.Series(dataset_title)
test["Title"].head()

# Convert to categorical values Title train
X["Title"] = X["Title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir',
                                 'Jonkheer', 'Dona'], 'Rare')
X["Title"] = X["Title"].map({"Master":0, "Miss":1, "Ms" : 1 , "Mme":1, "Mlle":1, "Mrs":1, "Mr":2, "Rare":3})
X["Title"] = X["Title"].astype(int)

# Convert to categorical values Title test
test["Title"] = test["Title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 
                                       'Jonkheer', 'Dona'], 'Rare')
test["Title"] = test["Title"].map({"Master":0, "Miss":1, "Ms" : 1 , "Mme":1, "Mlle":1, "Mrs":1, "Mr":2, "Rare":3})
test["Title"] = test["Title"].astype(int)

X.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Deck,Ticket_Frequency,Title
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,M,1,2
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,C,1,1
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,M,1,1
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,C,2,1
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,M,1,2


In [18]:
# Create Dummies
X['Embarked'] = X['Embarked'].fillna('S')

dummies = ['Pclass', 'Sex', 'Embarked', 'Title', 'Deck']
for dum in dummies:
    train_dummies = pd.get_dummies(X[dum], prefix = dum)
    test_dummies = pd.get_dummies(test[dum], prefix = dum)
    X = pd.concat([X, train_dummies], axis = 1)
    test = pd.concat([test, test_dummies], axis = 1)
print(X.shape, test.shape)

(891, 32) (418, 31)


In [19]:
X.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Deck,Ticket_Frequency,...,Title_3,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_M,Deck_T
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,M,1,...,0,0,0,0,0,0,0,0,1,0
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,C,1,...,0,0,0,1,0,0,0,0,0,0
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,M,1,...,0,0,0,0,0,0,0,0,1,0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,C,2,...,0,0,0,1,0,0,0,0,0,0
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,M,1,...,0,0,0,0,0,0,0,0,1,0


In [20]:
# Drop Columns
X.drop(labels = ['Pclass', 'Sex', 'Embarked', 'Name', 'Title', 'Deck'], axis = 1, inplace = True)
test.drop(labels =  ['Pclass', 'Sex', 'Embarked', 'Name', 'Title', 'Deck'], axis = 1, inplace = True)

In [21]:
X.head()

,Age,SibSp,Parch,Fare,Ticket_Frequency,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,...,Title_3,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_M,Deck_T
0,22.0,1,0,7.2500,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
1,38.0,1,0,71.2833,1,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,26.0,0,0,7.9250,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
3,35.0,1,0,53.1000,2,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
4,35.0,0,0,8.0500,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0


# Impute Age and fare for test

In [22]:
X.isna().any()

Age                  True
SibSp               False
Parch               False
Fare                False
Ticket_Frequency    False
Pclass_1            False
Pclass_2            False
Pclass_3            False
Sex_female          False
Sex_male            False
Embarked_C          False
Embarked_Q          False
Embarked_S          False
Title_0             False
Title_1             False
Title_2             False
Title_3             False
Deck_A              False
Deck_B              False
Deck_C              False
Deck_D              False
Deck_E              False
Deck_F              False
Deck_G              False
Deck_M              False
Deck_T              False
dtype: bool

In [23]:
test.isna().any()

Age                  True
SibSp               False
Parch               False
Fare                 True
Ticket_Frequency    False
Pclass_1            False
Pclass_2            False
Pclass_3            False
Sex_female          False
Sex_male            False
Embarked_C          False
Embarked_Q          False
Embarked_S          False
Title_0             False
Title_1             False
Title_2             False
Title_3             False
Deck_A              False
Deck_B              False
Deck_C              False
Deck_D              False
Deck_E              False
Deck_F              False
Deck_G              False
Deck_M              False
dtype: bool

In [24]:
df_train_test = pd.concat([X, test ], axis = 0)
df_train_test.shape

(1309, 26)

In [25]:
#imputer created an erroroneous data with "most_frequent" 
df_train_test['Age'] = df_train_test['Age'].fillna(df_train_test['Age'].value_counts().argmax())

imp = SimpleImputer(missing_values=np.nan, strategy='median')
#imp = IterativeImputer(random_state = SEED)
all_imp = imp.fit_transform(df_train_test)

col = X.columns.values
all_data = pd.DataFrame(all_imp, columns = col)

#discretize age and Fare
kbd_a = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='kmeans')
all_data['Age']  = kbd_a.fit_transform(all_data[['Age']])

kbd_f = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
all_data['Fare']  = kbd_f.fit_transform(all_data[['Fare']])

X = all_data.iloc[ :891,:]
test = all_data.iloc[891:,: ]
print(X.shape, test.shape)

(891, 26) (418, 26)


In [26]:
X.isna().any()

Age                 False
SibSp               False
Parch               False
Fare                False
Ticket_Frequency    False
Pclass_1            False
Pclass_2            False
Pclass_3            False
Sex_female          False
Sex_male            False
Embarked_C          False
Embarked_Q          False
Embarked_S          False
Title_0             False
Title_1             False
Title_2             False
Title_3             False
Deck_A              False
Deck_B              False
Deck_C              False
Deck_D              False
Deck_E              False
Deck_F              False
Deck_G              False
Deck_M              False
Deck_T              False
dtype: bool

In [27]:
test.isna().any()

Age                 False
SibSp               False
Parch               False
Fare                False
Ticket_Frequency    False
Pclass_1            False
Pclass_2            False
Pclass_3            False
Sex_female          False
Sex_male            False
Embarked_C          False
Embarked_Q          False
Embarked_S          False
Title_0             False
Title_1             False
Title_2             False
Title_3             False
Deck_A              False
Deck_B              False
Deck_C              False
Deck_D              False
Deck_E              False
Deck_F              False
Deck_G              False
Deck_M              False
Deck_T              False
dtype: bool

In [28]:
X.head()

,Age,SibSp,Parch,Fare,Ticket_Frequency,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,...,Title_3,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_M,Deck_T
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0
1,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0
4,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [29]:
Counter(y)

Counter({0: 549, 1: 342})

# Outliers

In [30]:
#studentized OLS for outliers
ols_mod = OLS(endog = y.values, exog = X)
ols_result = ols_mod.fit()

In [31]:
out_test = ols_result.outlier_test()['bonf(p)']

In [32]:
outliers = list(out_test[out_test<1e-3].index) 

outliers

[]

In [33]:
inf = OLSInfluence(ols_result)
stud = inf.resid_studentized_internal
less_outliers = list(stud[stud < -3.0 ].index)  #2
more_outliers = list(stud[stud > 3.0].index) 
print(less_outliers, more_outliers)

[] []


In [34]:
print(len(y.loc[more_outliers]))
#print(y.loc[more_outliers])
#print(X.loc[more_outliers])

0


In [35]:
print(len(y.loc[less_outliers]))
print(y.loc[less_outliers])

0
Series([], Name: Survived, dtype: int64)


In [36]:
drop_list = set(X.loc[less_outliers].index )
drop_list.update(X.loc[more_outliers].index)
print(len(drop_list))

0


In [37]:
X.drop(drop_list, axis = 0, inplace = True)
X.reset_index(drop = True, inplace = True)
y.drop(drop_list, axis = 0, inplace = True)
y = y.reset_index(drop = True)
print(X.shape, y.shape)

(891, 26) (891,)


In [38]:
Counter(y)

Counter({0: 549, 1: 342})

# Feature Engineering

In [39]:
#Family Total
X['tot_fam'] = X['Parch'] + X['SibSp']
test['tot_fam'] = test['Parch'] + test['SibSp']
X.head()

,Age,SibSp,Parch,Fare,Ticket_Frequency,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,...,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_M,Deck_T,tot_fam
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0
4,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [40]:
#Solo

X['solo'] = X['tot_fam'].apply(lambda x: 1 if x == 0 else 0 )
test['solo'] = test['tot_fam'].apply(lambda x: 1 if x == 0 else 0 )
X.head(10)

,Age,SibSp,Parch,Fare,Ticket_Frequency,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,...,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_M,Deck_T,tot_fam,solo
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1
1,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1
3,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,1
4,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
6,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,4.0,1.0,0.0,0.0,0.0,0.0,1
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,1
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,1


In [41]:
#Child
age_var = 10

X["Child"] = float('NaN')
test["Child"] = float('NaN')

X["Child"][X["Age"] < age_var] = 1
X["Child"][X["Age"] >= age_var] = 0

test["Child"][test["Age"] < age_var] = 1
test["Child"][test["Age"] >= age_var] = 0

#round age 
X['Age'] = X['Age'].apply(lambda x: custom_round(x, base=5))
test['Age'] = test['Age'].apply(lambda x: custom_round(x, base=5))
X.head()

,Age,SibSp,Parch,Fare,Ticket_Frequency,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,...,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_M,Deck_T,tot_fam,solo,Child
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1,1.0
1,0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1,1.0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1,1.0
3,0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,1,1.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1,1.0


In [42]:
test.head()

,Age,SibSp,Parch,Fare,Ticket_Frequency,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,...,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_M,Deck_T,tot_fam,solo,Child
891,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1,1.0
892,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1,1.0
893,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1,1.0
894,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1,1.0
895,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1,1.0


In [43]:
print(X.shape, test.shape)

(891, 29) (418, 29)


In [44]:
test.describe()

,Age,SibSp,Parch,Fare,Ticket_Frequency,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,...,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_M,Deck_T,tot_fam,solo,Child
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.0,...,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.0
mean,0.526316,0.016746,0.043062,0.167464,0.031100,0.021531,0.019139,0.002392,0.782297,0.0,...,0.636364,0.447368,1.349282,0.050239,0.361244,0.574163,0.014354,0.059809,0.940191,1.0
std,1.536300,0.128474,0.203240,0.554635,0.173797,0.145320,0.137177,0.048912,0.413179,0.0,...,0.481622,0.896760,0.757069,0.218700,0.480937,0.495062,0.119088,0.237416,0.237416,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,...,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.0
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,...,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.0
max,5.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,...,1.000000,8.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0


# Scaling

In [45]:
df_tt2 = pd.concat([X.copy(), test.copy() ], axis = 0)
df_tt2.reset_index(drop = True, inplace = True)
df_tt2.shape

(1309, 29)

In [46]:
length = len(X)
cols = X.columns.values

sc = preprocessing.MinMaxScaler()
scaled_cols = sc.fit_transform(df_tt2[['Age', 'SibSp', 'Parch', 'Fare']])


col = ['Age', 'SibSp', 'Parch', 'Fare']
sc_cols = pd.DataFrame(scaled_cols, columns = col)
sc_cols.reset_index(drop = True, inplace = True)
df_tt2.drop(labels =  ['Age', 'SibSp', 'Parch', 'Fare'], axis = 1, inplace = True)
df_tt3 = pd.concat([sc_cols, df_tt2], axis = 1, ignore_index = True)

X =    df_tt3.iloc[ :length,:]
X.columns = cols
test = df_tt3.iloc[length:,: ]
test.columns = cols
print(X.shape, test.shape)

(891, 29) (418, 29)


# Models

In [47]:
#Combine for tester
df_all = pd.concat([y, X ], axis = 1)
df_all.shape

(891, 30)

In [48]:
 # ols pipe smt
k_final = 16 #16
thresh_f = 0.61  #0.61

ols_m = SOLSClassifier(thresh_f, est_method = 'mean') #0.59 0.82978  0.82933 0.82822 0.61

model = Pipeline([  ('kBest', SelectKBest( chi2, k = k_final)), ('ols', ols_m)])
#model = ols_m
feat, dat = tester_prep(df_all)
test_classifier(model, dat, feat, folds = 50)

Pipeline(memory=None,
         steps=[('kBest',
                 SelectKBest(k=16,
                             score_func=<function chi2 at 0x000002422DAB4288>)),
                ('ols', SOLSClassifier(est_method='mean', threshold=0.61))],
         verbose=False)
	Accuracy: 0.80400	Precision: 0.77821	Recall: 0.69371	F1: 0.73353	F2: 0.70911
	Total predictions: 4500	True positives: 1214	False positives:  346	False negatives:  536	True negatives: 2404



In [49]:
ans = pd.read_csv(r'C:\Users\Me\Kaggle\Titanic_revisited\answers.csv')

In [50]:
k_final = 18#16, 25
thresh_f = 0.64  #0.61


ols_f = SOLSClassifier(thresh_f, est_method = 'mean')
k_mod = Pipeline([  ('kBest', SelectKBest( chi2, k = k_final)), ('ols', ols_f)])
k_mod.fit(X, y)
final_submit = k_mod.predict(test)


from sklearn.metrics import accuracy_score
y_pred =  final_submit
y_true =  ans['Survived'].values

score = accuracy_score(y_true, y_pred) #0.79425 = 0.78229 and .78947
print('Score: ', score,'\n', Counter(y_pred)) # 260/158

Score:  0.784688995215311 
 Counter({0: 276, 1: 142})


In [51]:
kb = SelectKBest(chi2, k='all')
kb.fit(X, y)

names = X.columns.values[kb.get_support()]
scores = kb.scores_[kb.get_support()]
names_scores = list(zip(names, scores))
ns_df = pd.DataFrame(data = names_scores, columns = ['Feature','f_score'])
ns_df_sorted = ns_df.sort_values(['f_score','Feature'], ascending =
[False, True])
print(ns_df_sorted)

             Feature      f_score
13           Title_0  4518.319091
18            Deck_B   170.348127
23            Deck_G   166.384570
24            Deck_M   112.805785
19            Deck_C    92.702447
15           Title_2    55.175151
17            Deck_A    41.553071
2              Parch    25.875581
26           tot_fam    22.594589
8         Sex_female    20.488314
10        Embarked_C    20.464401
4   Ticket_Frequency    19.489646
5           Pclass_1    18.140638
3               Fare    10.936730
14           Title_1    10.097499
22            Deck_F     6.180425
16           Title_3     6.160767
12        Embarked_S     5.489205
6           Pclass_2     2.946970
20            Deck_D     2.581865
27              solo     1.689825
21            Deck_E     1.349147
9           Sex_male     0.622951
1              SibSp     0.435116
0                Age     0.382745
7           Pclass_3     0.228214
25            Deck_T     0.126121
11        Embarked_Q     0.010847
28            

# Final Data

In [52]:
ols_f = SOLSClassifier(thresh_f, est_method = 'mean')
k_mod = Pipeline([  ('kBest', SelectKBest( chi2, k = k_final)), ('ols', ols_f)])
k_mod.fit(X, y)
final_submit = k_mod.predict(test)

In [53]:
from collections import Counter #{0: 259, 1: 159}{0: 260, 1: 158} //{0: 268, 1: 150}*{0: 260, 1: 158}
Counter(final_submit)

Counter({0: 276, 1: 142})

In [54]:
#Submission

my_solution = pd.DataFrame(final_submit, PassengerId, columns = ["Survived"])

my_solution.to_csv("solution.csv", index_label = ["PassengerId"])

In [55]:
#best was 38 need  ~377
my_solution['Survived'].mean()

0.3397129186602871

In [56]:
ans = pd.read_csv(r'C:\Users\Me\Kaggle\Titanic_revisited\answers.csv')

In [57]:
ans['Survived'].mean()

0.37799043062200954

In [58]:
Counter(ans['Survived'])

Counter({0: 260, 1: 158})

In [59]:
predictions =  my_solution['Survived'].values
labels_test =  ans['Survived'].values

true_negatives = 0
false_negatives = 0
true_positives = 0
false_positives = 0

for prediction, truth in zip(predictions, labels_test):
            if prediction == 0 and truth == 0:
                true_negatives += 1
            elif prediction == 0 and truth == 1:
                false_negatives += 1
            elif prediction == 1 and truth == 0:
                false_positives += 1
            elif prediction == 1 and truth == 1:
                true_positives += 1
            else:
                print ("Warning: Found a predicted label not == 0 or 1.")
                print ("All predictions should take value 0 or 1.")
                print ("Evaluating performance for processed predictions:")
                break
    
            total_predictions = true_negatives + false_negatives + false_positives + true_positives
            accuracy = (true_positives + true_negatives)/total_predictions
            #precision = 1.0*true_positives/(true_positives+false_positives)
            #recall = 1.0*true_positives/(true_positives+false_negatives)
            #f1 = 2.0 * true_positives/(2*true_positives + false_positives+false_negatives)
            #f2 = (1+2.0*2.0) * precision*recall/(4*precision + recall)
print(accuracy)

0.784688995215311


In [60]:
# 0.79425 = 0.78229 and .78947
